In [2]:
#importing neccessary libraries
import os
import pickle
import tensorflow as tf
from flask import Flask, render_template, request, redirect, url_for

from glob import glob
import librosa as lr
import librosa.display
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [3]:
app = Flask(__name__)

In [4]:
#loading model and encoder objects
with open('./artefacts/finalized_model.pkl','rb') as model_file:
     urban_model = pickle.load(model_file)

with open('./artefacts/encoder.pkl','rb') as model_file:
     label_en = pickle.load(model_file)

Using TensorFlow backend.


In [5]:
#for keras -- to use default graph
graph = tf.get_default_graph()

In [6]:
#current location of the flask application
APP_ROOT = os.path.dirname(os.path.realpath('__file__'))

In [7]:
#extracting sound features of the input sound file
from glob import glob
import librosa as lr
import librosa.display
import numpy as np

def preprocess(file):
    x,s=librosa.load(file,res_type='kaiser_fast')
    stft = np.abs(librosa.stft(x))
    mfccs=np.mean(librosa.feature.mfcc(y=x,sr=s,n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=s, n_chroma=40).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(x, sr=s, n_mels=40).T,axis=0)
    Xtest=np.vstack((mfccs,chroma,mel)).T
    X_2dt=np.reshape(Xtest,(1,Xtest.shape[0]*Xtest.shape[1]))
    return X_2dt

In [8]:
#index page function
@app.route('/')
def index():
    return render_template('new1.html')

In [9]:
#upload and predict function
@app.route("/upload", methods=['GET','POST'])
def upload():
    
    global graph
    with graph.as_default():
        target = os.path.join(APP_ROOT, 'sounds/')
        print(target)

        if not os.path.isdir(target):
            os.mkdir(target)

        for file in request.files.getlist("file"):
            print(file)
            filename = file.filename
            destination = "/".join([target, filename])
            print(destination)
            file.save(destination)
        
            X_final = preprocess(destination)
            print(label_en.inverse_transform(urban_model.predict_classes(X_final)))
            sound_result = label_en.inverse_transform(urban_model.predict_classes(X_final))
            
            try:
            
                if sound_result == 'gun_shot':
                    level = "Stay indoors.Call for help"
                else:
                    level = "Relax! It's nothing to worry"
             
            except:
                level = ""

    return render_template('complete1.html', Result=sound_result, Level=level)


In [10]:
#run app
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2019 13:15:33] "GET / HTTP/1.1" 200 -


C:\Users\Harini Pravin\Downloads\urban-sound-classification\flaskapp\sounds/
<FileStorage: '5.wav' ('audio/wav')>
C:\Users\Harini Pravin\Downloads\urban-sound-classification\flaskapp\sounds//5.wav


127.0.0.1 - - [29/May/2019 13:15:45] "POST /upload HTTP/1.1" 200 -


['drilling']


...

['drilling']


127.0.0.1 - - [29/May/2019 15:17:18] "GET /? HTTP/1.1" 200 -


C:\Users\Harini Pravin\Downloads\urban-sound-classification\flaskapp\sounds/
<FileStorage: '19.wav' ('audio/wav')>
C:\Users\Harini Pravin\Downloads\urban-sound-classification\flaskapp\sounds//19.wav


127.0.0.1 - - [29/May/2019 15:17:24] "POST /upload HTTP/1.1" 200 -


['gun_shot']
